# Анализ метрик AIDA для Youtube канала

Бизнес все чаще использует контент-маркетинг для своего продвижения. Становится тяжелее отслеживать результаты, тренды и удачные примеры. Какой контент хочет видеть наша аудитория? О чем и как снимать? Какие результаты у нашего канала? Как это влияет на наш бизнес? Несмотря на встроенные возможности Youtube для аналитики каналов (Youtube Studio) - возможности этой платформы зачастую ограничены или не обладают необходимой для бизнеса гибкость.

**Цель исследования** - Разработать автоматическую систему анализа YouTube-канала для бизнеса. Это система должна помогать:
владельцу бизнеса и стейкхолдерам отслеживать эффективность бизнес-единицы (YouTube-канала) и позволять интегрировать данные бизнеса для расчёта дополнительных параметров (например, добавление информации о расходах для расчёта метрики **ROI/ROMI**)

**Ход исследования**

Заказчик - *Роман Сергеев*, создатель Youtube-канала с рисованными видео на книги по бизнесу, психологии и личной эффективности [**RomanSergeevCom**](https://www.youtube.com/c/RomanSergeevCom/about), на момент исследования на канал подписаны более 485 тыс. человек. Основными источниками данных станут выгрузка общей статистики канала за октябрь 2022 с разбивкой по дням и статистика всех видео канала за этот же период. Будет проведён обзор и предобработка данных - данные будут проверены на наличие дубликатов, пропущенных или аномальных значений, а также корректность типа хранимых данных. После чего для данных будут расчитаны метрики по модели [**AIDA**](https://en.wikipedia.org/wiki/AIDA_(marketing)) (согласовано с заказчиком), но основе которых будет проведён исследовательский анализ, в ходе которого будет изучено распределение значений метрик в наборах данных. По результатам будет сделан общий вывод, после чего датасеты будут выгружены для создания интерактивного дашборда.

Таким образом, исследование будет разбито на следующие этапы:


1. [**Обзор и предобработка данных**](#1)
2. [**Исследовательский анализ данных**](#2)
3. [**Общий вывод**](#3)

Для выполнения проекта будет использованы следующие инструменты:

- **Предобработка/исследовательский анализ** - библиотеки для анализа и визуализации (pandas/numpy/matplotlib/seaborn)
- **Дашборд** - Tableau Public

<a id='1'></a>
## Обзор и предобработка данных
На данном этапе данные будут предварительно изучены, после чего будут проверены на наличие дубликатов, пропущенных или аномальных значений, а также корректность типа хранимых данных.

In [1]:
# импортируем библиотеки для работы с данными

import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
from matplotlib import pyplot as plt

# настройки отображения

pd.options.display.float_format = '{:,.2f}'.format
pd.options.mode.chained_assignment = None
sns.set_style('darkgrid')
plt.rcParams.update({'axes.labelsize': 12,'axes.titlesize': 16})
plt.rcParams['figure.figsize'] = [15, 5]

# путь к датасету

import os
os.chdir('C:\\Users\\dmitr\\GitHub\\Pet-Projects\\datasets')

### Общая статистика канала

In [2]:
# загружаем первый датафрейм (общая статистика канала), выводим общую информацию и 10 случайных строк
# первая строка в сырых данных содержит итоги столбцов, исключим её из выгрузки

overall = pd.read_excel('youtube_overall_october.xlsx').drop(index=0)

overall.info()
display(overall.sample(10, random_state=0).T)
print(f'Датафрейм содержит {overall.shape[1]} столбца и {overall.shape[0]} строк')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 1 to 31
Data columns (total 24 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Date                                     31 non-null     object 
 1   Clicks per end screen element shown (%)  31 non-null     float64
 2   End screen element clicks                31 non-null     float64
 3   Teaser clicks per card teaser shown (%)  31 non-null     float64
 4   Card teaser clicks                       31 non-null     float64
 5   Clicks per card shown (%)                28 non-null     float64
 6   Card clicks                              31 non-null     float64
 7   Comments added                           31 non-null     float64
 8   Shares                                   31 non-null     float64
 9   Dislikes                                 31 non-null     float64
 10  Likes                                    31 non-null

,3,30,14,11,28,26,23,12,18,24
Date,2022-10-01,2022-10-24,2022-10-03,2022-10-17,2022-10-11,2022-10-04,2022-10-08,2022-10-25,2022-10-26,2022-10-18
Clicks per end screen element shown (%),2.29,1.23,1.90,2.03,1.38,1.45,1.62,1.96,1.75,1.60
End screen element clicks,33.00,20.00,31.00,39.00,25.00,43.00,20.00,28.00,26.00,29.00
Teaser clicks per card teaser shown (%),0.54,0.14,0.25,0.00,0.74,0.00,0.00,0.16,0.27,0.29
Card teaser clicks,3.00,1.00,2.00,0.00,5.00,0.00,0.00,1.00,2.00,2.00
Clicks per card shown (%),25.00,0.00,25.00,0.00,14.29,0.00,NaN,33.33,20.00,16.67
Card clicks,2.00,0.00,1.00,0.00,1.00,0.00,0.00,1.00,1.00,1.00
Comments added,1.00,4.00,3.00,2.00,3.00,11.00,5.00,2.00,0.00,1.00
Shares,65.00,109.00,74.00,85.00,107.00,111.00,95.00,82.00,111.00,123.00
Dislikes,1.00,-1.00,6.00,2.00,0.00,8.00,2.00,5.00,5.00,3.00


Датафрейм содержит 24 столбца и 31 строк


Описание данных:

- `Clicks per end screen element shown (%)` - отношение нажатий на элемент конечной заставки к его показам;
- `End screen element clicks` - число нажатий на элемент конечной заставки;
- `Teaser clicks per card teaser shown (%)` - отношение количества кликов по тизеру к общему числу его показов;
- `Card teaser clicks` -  количество кликов по тизеру;
- `Clicks per card shown (%)` - отношение количества кликов по подсказке к общему числу ее показов;
- `Card clicks` - клики на подсказку;
- `Comments added` - количество новых комментариев;
- `Shares` - количество нажатий на кнопку "Поделиться";
- `Dislikes` - количество нажатий на кнопку "Не нравится";
- `Likes` - количество нажатий на кнопку "Нравится";
- `Subscribers gained` - общее число пользователей, подписавшихся на этот канал за выбранный период времени;
- `Returning viewers` - вернувшиеся зрители;
- `New viewers` - новые зрители;
- `Unique viewers` - уникальные зрители;
- `Impressions click-through rate (%)` - как часто пользователи просматривают видео после того, как увидят его значок;
- `Impressions` - показы;
- `Videos published` - количество опубликованных видео;
- `Videos added` - количество добавленных видео;
- `Subscribers` - количество подписчиков;
- `Average percentage viewed (%)` - средний процент просмотров (%);
- `Views` - просмотры;
- `Watch time (hours)` - время просмотра (часы);
- `Average view duration` - средняя продолжительность просмотра.


Приведём названия колонок к единому стилю и преобразуем даты:

In [3]:
# меняем названия колонок

overall.columns = [
    x
    .lower()
    .replace(' ' , '_')
    .replace('-', '_')
    .replace('(', '')
    .replace(')', '')
    for x in overall.columns
]

# преобразуем даты

overall['date'] = pd.to_datetime(overall['date'])

Изучим описательную статистику таблицы:

In [4]:
overall.describe().T

,count,mean,std,min,25%,50%,75%,max
clicks_per_end_screen_element_shown_%,31.00,1.84,0.36,1.23,1.61,1.77,2.15,2.55
end_screen_element_clicks,31.00,30.45,7.00,18.00,26.00,30.00,35.50,46.00
teaser_clicks_per_card_teaser_shown_%,31.00,0.28,0.23,0.00,0.15,0.25,0.36,0.75
card_teaser_clicks,31.00,1.84,1.53,0.00,1.00,2.00,2.00,6.00
clicks_per_card_shown_%,28.00,12.18,14.23,0.00,0.00,6.67,20.00,50.00
card_clicks,31.00,0.55,0.68,0.00,0.00,0.00,1.00,2.00
comments_added,31.00,2.06,2.24,0.00,1.00,2.00,2.50,11.00
shares,31.00,90.84,18.35,59.00,74.50,90.00,107.00,128.00
dislikes,31.00,2.68,2.52,-1.00,1.00,2.00,5.00,8.00
likes,31.00,110.29,37.48,57.00,88.00,109.00,124.50,277.00


Сохраним данные для дашборда:

In [8]:
# записываем датафрейм обратно в .csv

overall.to_csv(os.getcwd() + '/overall_processed.csv')

### Статистика по видео

In [5]:
# загружаем второй датафрейм (статистика по каждому видео), выводим общую информацию и 10 случайных строк
# первая строка в сырых данных содержит итоги столбцов, исключим её из выгрузки

content = pd.read_excel('youtube_content_october.xlsx').drop(index=0)

content.info()
display(content.sample(10, random_state=0).T)
print(f'Датафрейм содержит {content.shape[1]} столбца и {content.shape[0]} строк')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 220 entries, 1 to 220
Data columns (total 24 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Content                                  220 non-null    object 
 1   Video title                              220 non-null    object 
 2   Video publish time                       191 non-null    object 
 3   Clicks per end screen element shown (%)  178 non-null    float64
 4   End screen element clicks                195 non-null    float64
 5   Teaser clicks per card teaser shown (%)  176 non-null    float64
 6   Card teaser clicks                       195 non-null    float64
 7   Card clicks                              195 non-null    float64
 8   Clicks per card shown (%)                39 non-null     float64
 9   Subscribers gained                       195 non-null    float64
 10  Unique viewers                           220 non-n

,153,75,72,162,163,144,64,154,220,136
Content,DFLRIY0ir1o,-szfQJ1-9OM,1HFRquFwhIM,erxfLffk8IQ,Sd97h4xhepo,z41Du_6vDHI,APZqHC1Jo0s,fUPNozpFav8,ygIEgoCZFnA,BJuRCUn5Xt4
Video title,«Конор Макгрегор». Джон Кавана | Саммари ®,«Привычка достигать». Бернард Рос | Саммари,«Первоклассный сервис как конкурентное преимущ...,Взрывной рост компании «Кнопка»,«Отдел продаж по захвату рынка». Часть 3. Миха...,«Уоррен Баффет» Элис Шрёдер | Саммари,«Красная Таблетка» Андрей Курпатов | (АНИМАЦИЯ),«Золотая лихорадка». Владислав Мусатов | Самма...,Antonio 07 Acoustic | Music by RomanSergeevCom,«Экономика впечатлений» Джозеф Пайн и Джеймс Г...
Video publish time,"Oct 4, 2018","Sep 12, 2017","Jul 28, 2021","Dec 28, 2019","Oct 31, 2019","Jan 23, 2018","May 26, 2018","Feb 13, 2019",NaN,"Oct 6, 2019"
Clicks per end screen element shown (%),NaN,1.56,1.49,0.00,0.00,0.00,2.06,0.00,NaN,3.09
End screen element clicks,0.00,2.00,3.00,0.00,0.00,0.00,7.00,0.00,NaN,3.00
Teaser clicks per card teaser shown (%),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00
Card teaser clicks,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00
Card clicks,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00
Clicks per card shown (%),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Subscribers gained,1.00,4.00,3.00,0.00,0.00,0.00,2.00,4.00,NaN,2.00


Датафрейм содержит 24 столбца и 220 строк


Описание данных:

- `Content` - идентификатор видео;
- `Video title` - название видео;
- `Video publish time` - время публикации видео;
- `Clicks per end screen element shown (%)` - отношение нажатий на элемент конечной заставки к его показам;
- `End screen element clicks` - число нажатий на элемент конечной заставки;
- `Teaser clicks per card teaser shown (%)` - оЭтношение количества кликов по тизеру к общему числу его показов;
- `Card teaser clicks` -  количество кликов по тизеру;
- `Card clicks` - клики на подсказку;
- `Clicks per card shown (%)` - отношение количества кликов по подсказке к общему числу ее показов;

- `Subscribers gained` - общее число пользователей, подписавшихся на этот канал за выбранный период времени;
- `Unique viewers` - уникальные зрители;
- `Returning viewers` - вернувшиеся зрители;
- `New viewers` - Новые зрители;
- `Average percentage viewed (%)` - Средний процент просмотров (%);
- `Subscribers` - количество подписчиков;
- `Comments added` - количество новых комментариев;
- `Shares` - количество нажатий на кнопку "Поделиться";
- `Dislikes` - количество нажатий на кнопку "Не нравится";
- `Likes` - количество нажатий на кнопку "Нравится";
- `Views` - просмотры;
- `Watch time (hours)` - время просмотра (часы);
- `Average view duration` - средняя продолжительность просмотра.
- `Impressions` - показы;
- `Impressions click-through rate (%)` - как часто пользователи просматривают видео после того, как увидят его значок;

Приведём названия колонок к единому стилю и преобразуем даты:

In [6]:
# меняем названия колонок

content.columns = [
    x
    .lower()
    .replace(' ' , '_')
    .replace('-', '_')
    .replace('(', '')
    .replace(')', '')
    for x in content.columns
]

# преобразуем даты

content['video_publish_time'] = pd.to_datetime(content['video_publish_time'])

Изучим описательную статистику таблицы:

In [7]:
content.describe().T

,count,mean,std,min,25%,50%,75%,max
clicks_per_end_screen_element_shown_%,178.00,1.66,1.94,0.00,0.00,1.17,2.21,11.59
end_screen_element_clicks,195.00,4.84,9.50,0.00,0.00,1.00,5.00,62.00
teaser_clicks_per_card_teaser_shown_%,176.00,0.32,0.92,0.00,0.00,0.00,0.00,6.47
card_teaser_clicks,195.00,0.29,1.08,0.00,0.00,0.00,0.00,13.00
card_clicks,195.00,0.09,0.55,0.00,0.00,0.00,0.00,7.00
clicks_per_card_shown_%,39.00,10.23,21.46,0.00,0.00,0.00,5.55,100.00
subscribers_gained,195.00,7.51,18.34,0.00,0.00,2.00,6.00,151.00
unique_viewers,220.00,400.65,861.37,0.00,41.75,156.50,352.50,"6,807.00"
returning_viewers,220.00,149.30,352.25,0.00,18.00,60.50,149.25,"4,154.00"
new_viewers,220.00,251.35,574.62,0.00,19.00,85.50,220.25,"4,983.00"


Сохраним данные для дашборда:

In [10]:
# записываем датафрейм обратно в .csv

content.to_csv(os.getcwd() + '/content_processed.csv')